In [ ]:
# import all necessary libs
import requests
import pandas as pd
import matplotlib.pyplot as plot
from arcgis.features import FeatureLayer
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
# a function connects to the UToronto ArcGIS server and return the connection object
# this function takes only one parameter - cid which represents the client_id 
def get_connect(cid=""):
    # print warning if no client id is provided
    if not cid:
        print("empty client id, please enter a valid client id to continue...")
    else:
        gis = GIS(url="https://utoronto.maps.arcgis.com",client_id=cid)
        print("Successfully logged in as: " + gis.properties.user.username)
        return gis

In [ ]:
print("Please enter a valid client id to continue:")
client_id = input()
# execute get_connect function to create connection object 
my_gis = get_connect(client_id)

In [ ]:
# get the desired item by doing a search with title and owner as a filter
item = my_gis.content.search(
    query="title:Coronavirus COVID-19 Cases V2 owner:CSSE_covid19", 
    item_type="Feature Layer",
    outside_org=True)[0]
item

In [ ]:
# get the feature layer object
fl = FeatureLayer(url=item.url + "/1")

In [ ]:
map1 = my_gis.map("Australia")

map1.add_layer(fl, { "type": "FeatureLayer",
                     "renderer":"ClassedSizeRenderer",
                     "field_name":"Confirmed"})

map1.zoom=3 # size to zoom
map1.legend=True # show the legend

map1

In [ ]:
# get the Australia DataFrame
my_df = fl.query(where="Country_Region='Australia'", as_df=True)

In [ ]:
map2 = my_gis.map("Australia")

my_df.spatial.plot(kind='map', 
                   map_widget=map2,
                   renderer_type='c',
                   method='esriClassifyStandardDeviation',
                   class_count=8,
                   col='Deaths',
                   cmap='viridis',
                   alpha=0.6,
                  )

map2.zoom = 3 # size to zoom
map2.legend = True # show the legend

map2

In [ ]:
# a brief summary of the latest updated cases per province of Australia
df_australia = my_df[["Province_State", "Last_Update", "Confirmed", "Recovered", "Deaths"]]
df_australia

In [ ]:
# A function returns the province with the largest value for a given column name
def get_largest_prov(df, col):
    # invalid column name, print error message
    if col not in df:
        print("The given column " + col + "does not exist")
    else:
        return df.nlargest(1, [col])[["Province_State", col]] # Pandas method to get n largest values for the given column
    
# A function returns the province with the smallest value for a given column name
def get_smallest_prov(df, col):
    # invalid column name, print error message
    if col not in df:
        print("The given column does not exist")
    else:
        return df.nsmallest(1, [col])[["Province_State", col]] # Pandas method to get n smallest values for the given column

In [ ]:
print("The province with the highest confirmed cases in the latest update")
get_largest_prov(df_australia, "Confirmed")

In [ ]:
print("The province with the lowest confirmed cases in the latest update")
get_smallest_prov(df_australia, "Confirmed")

In [ ]:
print("The province with the highest deaths in the latest update")
get_largest_prov(df_australia, "Deaths")

In [ ]:
print("The province with the lowest deaths in the latest update")
get_smallest_prov(df_australia, "Deaths")

In [ ]:
# a bar chart to display number of confirmed and deaths in each province in the latest update

bar = df_australia[["Province_State", "Confirmed", "Deaths"]].plot.bar(x="Province_State", 
                                                                       rot=90, 
                                                                       title="Number of Confirmed / Deaths per Province");

plot.show(block=True)

In [ ]:
# a scatter chart to display number of deaths in each province in the latest update

scatter = df_australia.plot.scatter(x="Province_State",
                                    y='Deaths',
                                    c='Black', 
                                    rot=90, 
                                    title="Number of Deaths per Province")

In [ ]:
# a pie chart to display number of confirmed in each province in the latest update

# we only need 2 columns - Province_State and Confirmed
df_pie = df_australia[["Province_State", "Confirmed"]]

# make column Province_State as the index of the sub DataFrame
df_pie.index = df_pie["Province_State"]

# plot the chart
pie = df_pie.plot.pie(y="Confirmed", figsize=(6, 6), title="Number of Confirmed per Province")